# 🧠 Stage 7 — BM25 + *Query Understanding* (requête enrichie)

**But :** comparer un **BM25 sur requête brute** vs un **BM25 sur requête enrichie métier** (dictionnaire juridique), puis mesurer l'impact via **Recall@10**, **MRR** et **nDCG@10**.

---

## ✅ À avoir dans le projet
- `corpus_loader.py` (ou package) → fournit `documents`
- `query_understanding.py` → `process_user_query`, `load_juridical_dictionary`
- `benchmark_queries.py` → `benchmark_queries`
- `juridical_dictionary.yml` → dictionnaire métier
- (optionnel) `metrics.py` → fonctions de métriques


In [1]:
# ▶️ Dépendances "pip" (uniquement celles qui sont externes au projet)
# Note : les modules `corpus_loader`, `query_understanding`, `benchmark_queries`, `metrics`
# sont supposés être des fichiers .py DU PROJET (donc pas installables via pip).

import sys, os
print("Python:", sys.version)

# Installation dans l'environnement Jupyter courant
# (relance la cellule si besoin après un redémarrage du kernel)
%pip -q install rank_bm25 pyyaml


Python: 3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 🔧 Rendre importables les modules locaux du projet
# Si notre notebook est dans un sous-dossier (ex: "notebooks/"), on ajoute la racine du projet au PYTHONPATH.

from pathlib import Path
import sys

# Point de départ : dossier du notebook
HERE = Path.cwd()

# Cas fréquent : notebook dans "notebooks/" → racine = parent
# Ajuste si besoin (parent.parent, etc.)
PROJECT_ROOT = HERE
if (HERE / "notebooks").exists() and not (HERE / "corpus_loader.py").exists():
    PROJECT_ROOT = HERE.parent

# On ajoute au sys.path en priorité (index 0)
sys.path.insert(0, str(PROJECT_ROOT))

print("Notebook folder :", HERE)
print("Project root    :", PROJECT_ROOT)
print("corpus_loader.py:", (PROJECT_ROOT / "corpus_loader.py").exists())


Notebook folder : d:\-- Projet RAG Avocats --\codes_python\notebooks
Project root    : d:\-- Projet RAG Avocats --\codes_python\notebooks
corpus_loader.py: True


In [3]:
# 📦 Imports
from rank_bm25 import BM25Okapi
import re

# Imports "locaux projet" : on affiche une erreur claire si un fichier manque
try:
    from corpus_loader import documents
except ModuleNotFoundError as e:
    raise ModuleNotFoundError(
        "Impossible d'importer `corpus_loader`. "
        "Vérifie que `corpus_loader.py` est bien dans PROJECT_ROOT (ou ajuste la cellule sys.path)."
    ) from e

try:
    from query_understanding import process_user_query, load_juridical_dictionary
except ModuleNotFoundError as e:
    raise ModuleNotFoundError(
        "Impossible d'importer `query_understanding`. "
        "Vérifie que `query_understanding.py` est bien dans PROJECT_ROOT."
    ) from e

try:
    from benchmark_queries import benchmark_queries
except ModuleNotFoundError:
    # Fallback : mini jeu de requêtes pour que le notebook tourne quand même
    benchmark_queries = [
        {"question": "Quelles sont les conditions de la garde à vue ?", "relevant_keywords": ["garde", "vue"]},
        {"question": "Délai de prescription en matière civile", "relevant_keywords": ["prescription", "délai"]},
    ]
    print("⚠️ `benchmark_queries` introuvable → fallback minimal utilisé (pour démo).")

# Metrics : soit un module local, soit un fallback minimal
try:
    from metrics import recall_at_k, reciprocal_rank, ndcg_at_k
except ModuleNotFoundError:
    print("⚠️ `metrics` introuvable → fallback minimal utilisé (pour démo).")

    def _is_relevant(doc_text: str, relevant_keywords) -> bool:
        t = (doc_text or "").lower()
        return any((kw or "").lower() in t for kw in relevant_keywords)

    def recall_at_k(results, relevant_keywords, k: int = 10) -> float:
        topk = results[:k]
        return 1.0 if any(_is_relevant(doc.get("text", ""), relevant_keywords) for doc, _ in topk) else 0.0

    def reciprocal_rank(results, relevant_keywords) -> float:
        for i, (doc, _) in enumerate(results, start=1):
            if _is_relevant(doc.get("text", ""), relevant_keywords):
                return 1.0 / i
        return 0.0

    def ndcg_at_k(results, relevant_keywords, k: int = 10) -> float:
        # Gain binaire : 1 si doc pertinent, sinon 0
        gains = [1.0 if _is_relevant(doc.get("text", ""), relevant_keywords) else 0.0
                 for doc, _ in results[:k]]

        def dcg(vals):
            s = 0.0
            for i, v in enumerate(vals, start=1):
                s += v / ( (i + 1) ** 0.5 )  # discount doux (démo)
            return s

        ideal = sorted(gains, reverse=True)
        denom = dcg(ideal)
        return (dcg(gains) / denom) if denom > 0 else 0.0


Corpus brut chargé : 4422 documents
Corpus filtré 'Code du travail' : 882 documents


In [4]:
# 📚 Chargement du dictionnaire juridique
# Assure-toi que `juridical_dictionary.yml` est bien présent à la racine du projet.
dictionary_path = "juridical_dictionary.yml"
dictionary = load_juridical_dictionary(dictionary_path)
print("Dico chargé ✅")


Dico chargé ✅


## 🔤 1) Tokenisation (simple & robuste)

In [5]:
def tokenize(text: str):
    # Tokenisation simple : mots alphanumériques en minuscules
    return re.findall(r"\b\w+\b", (text or "").lower())

# Petit test
tokenize("Article L. 123-4 : délai de prescription ?")


['article', 'l', '123', '4', 'délai', 'de', 'prescription']

## 🧱 2) Construction de l’index BM25

In [6]:
# On tokenise les documents (attendu : documents = [{"text": ...}, ...])
tokenized_docs = [tokenize(doc.get("text", "")) for doc in documents]

bm25 = BM25Okapi(tokenized_docs)

def bm25_search(query: str, k: int = 10):
    # Calcule les scores BM25 et retourne les top-k
    scores = bm25.get_scores(tokenize(query))
    ranked = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)
    return ranked[:k]

print("BM25 prêt ✅ - nb docs:", len(documents))


BM25 prêt ✅ - nb docs: 882


## 🧪 3) Évaluation : requête brute vs requête enrichie

In [7]:
def evaluate_bm25(use_query_understanding: bool = False, k: int = 10):
    recall_scores = []
    mrr_scores = []
    ndcg_scores = []

    for q in benchmark_queries:
        query = q["question"]

        # Enrichissement "métier"
        if use_query_understanding:
            enriched = process_user_query(query, dictionary)
            query = enriched.get("enriched_query", query)

        results = bm25_search(query, k=k)

        recall_scores.append(recall_at_k(results, q["relevant_keywords"], k))
        mrr_scores.append(reciprocal_rank(results, q["relevant_keywords"]))
        ndcg_scores.append(ndcg_at_k(results, q["relevant_keywords"], k))

    n = max(len(benchmark_queries), 1)
    return {
        f"Recall@{k}": sum(recall_scores) / n,
        "MRR": sum(mrr_scores) / n,
        f"nDCG@{k}": sum(ndcg_scores) / n,
    }

baseline = evaluate_bm25(use_query_understanding=False, k=10)
enriched = evaluate_bm25(use_query_understanding=True, k=10)

baseline, enriched


({'Recall@10': 0.3333333333333333,
  'MRR': 0.3333333333333333,
  'nDCG@10': 0.3333333333333333},
 {'Recall@10': 0.3333333333333333,
  'MRR': 0.3333333333333333,
  'nDCG@10': 0.3333333333333333})

## 📊 4) Comparaison lisible

In [8]:
import pandas as pd

df = pd.DataFrame([baseline, enriched], index=["BM25 requête brute", "BM25 requête enrichie"])
df


,Recall@10,MRR,nDCG@10
BM25 requête brute,0.333333,0.333333,0.333333
BM25 requête enrichie,0.333333,0.333333,0.333333


## 🔎 5) Inspection qualitative sur une requête

In [ ]:
# Modifier la requête pour tester rapidement
query = benchmark_queries[0]["question"]

print("=== Requête brute ===")
for i, (doc, score) in enumerate(bm25_search(query, k=5), start=1):
    print(f"{i:02d}  score={score:.4f}  |  {doc.get('id', '(no id)')}")

print("\n=== Requête enrichie ===")
enriched_q = process_user_query(query, dictionary).get("enriched_query", query)
print("Enriched query:", enriched_q)

for i, (doc, score) in enumerate(bm25_search(enriched_q, k=5), start=1):
    print(f"{i:02d}  score={score:.4f}  |  {doc.get('id', '(no id)')}")


=== Requête brute ===
01  score=23.5373  |  (no id)
02  score=12.7041  |  (no id)
03  score=12.6815  |  (no id)
04  score=12.6591  |  (no id)
05  score=12.2068  |  (no id)

=== Requête enrichie ===
Enriched query: Dans quels cas un CDI peut-il être rompu sans préavis ?
01  score=23.5373  |  (no id)
02  score=12.7041  |  (no id)
03  score=12.6815  |  (no id)
04  score=12.6591  |  (no id)
05  score=12.2068  |  (no id)


In [ ]:
res = process_user_query(query, dictionary)
print(res["intent_detected"]) # si None explique pourquoi requete ET requete enrichie donnent le même résultat
print(res["notes"] if "notes" in res else "")
print(res["enriched_query"])


None
Aucune intention métier détectée
Dans quels cas un CDI peut-il être rompu sans préavis ?


## 🧩 STAGE 5 — BM25 + “Query Understanding” : analyse finale (avec le diagnostic intention=None)

### 🧾 Données & périmètre (inchangés)
- Corpus brut : **4422** documents  
- Corpus filtré **« Code du travail »** : **882** documents  
✅ Même périmètre que les stages précédents → comparaison directe.

---

## 📊 Résultats (métriques)
### BM25 — requête brute
- Recall@10 = **0.333**
- MRR = **0.333**
- nDCG@10 = **0.333**

### BM25 — requête enrichie “métier”
- Recall@10 = **0.333**
- MRR = **0.333**
- nDCG@10 = **0.333**

➡️ Lecture : aucune amélioration mesurable sur ce benchmark.

---

## 🔍 Diagnostic clé (pour expliquer pourquoi c’est identique)
Sur la requête testée, le module de “query understanding” renvoie :

- `intent_detected = None`
- message : **« Aucune intention métier détectée »**
- `enriched_query` = **requête d’origine** (inchangée)

✅ Conséquence directe : comme la requête envoyée à BM25 est **strictement la même**,  
le ranking et les scores BM25 sont **strictement identiques** :

- Top-5 scores identiques (23.5373, 12.7041, 12.6815, 12.6591, 12.2068)
- Donc aucune variation possible sur Recall@10 / MRR / nDCG@10.

➡️ Conclusion importante : **l’absence de gain ne reflète pas un échec de BM25**,  
mais le fait que la couche “métier” **n’a pas été activée** sur cette question.

---

## 🧠 Comparaison avec les stages précédents
- **STAGE 3 (BM25 filtré)** : baseline = **0.333 / 0.333 / 0.333**
- **STAGE 4 (Dense)** : **0.333 / 0.333 / 0.292** (ranking plus bruité)
- **STAGE 5 (BM25 + enrichissement)** : **identique** à STAGE 3, car `intent=None` → pas d’enrichissement

---

## ✅ Conclusion
À ce stade, le STAGE 5 montre surtout un point “ingénierie” essentiel :

> Tant que l’intention métier n’est pas détectée (`intent=None`),
> la requête enrichie = requête brute,
> donc aucune amélioration n’est possible.

La prochaine étape logique (sans changer la logique BM25) serait de **renforcer la détection d’intention** et/ou le dictionnaire, afin que l’enrichissement se déclenche réellement et puisse être évalué objectivement.
